In [465]:
import tempfile
import pandas as pd
pd.set_option('display.max_rows', 100)

import numpy as np
import os
from tqdm import tqdm
from glob import glob
from datetime import timedelta
import datetime

In [466]:
def get_ranges(df, col):
    temp = []
    return_array = []
    for i in range(df.shape[0]):
        if(df.loc[i, col] == 1):
            temp.append(i)

            if i+1 < df.shape[0] and df.loc[i+1, col] != 1:
                return_array.append(temp)
                temp = []
                continue;
    return return_array

In [476]:
# Data from 
# https://github.com/IntelLabs/TSAD-Evaluator/tree/master/examples/simple

df_pred = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/simple/simple.pred")
df_real = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/simple/simple.real")


"""
Below I tried to calculate classical_recall_T for LSTM-AD testing over the 7 datasets and compare with the recall values in the (figure3.a)
The assumptions based on the article: alpha=0, bias=1(flat), gamma=1

"""
# # Twitter AAPL-lstm data
# df_pred = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/aapl/lstm_ad.pred")
# df_real = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/aapl/lstm_ad.real")

# timeguided_lstm data
# df_pred = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/timeguided/lstm_ad.pred")
# df_real = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/timeguided/lstm_ad.real")

# # ecg_lstm data
# df_pred = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/ecg/lstm_ad.pred")
# df_real = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/ecg/lstm_ad.real")

# # spaceshuttle_lstm data
# df_pred = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/space_shuttle/lstm_ad.pred")
# df_real = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/space_shuttle/lstm_ad.real")

# # sine_lstm data
# df_pred = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/sine/lstm_ad.pred")
# df_real = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/sine/lstm_ad.real")

# # nyc taxi_lstm data
# df_pred = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/nyc_taxi/lstm_ad.pred")
# df_real = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/nyc_taxi/lstm_ad.real")

# # machine_temp_lstm data
# df_pred = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/machine_temp/lstm_ad.pred")
# df_real = pd.read_csv("https://raw.githubusercontent.com/IntelLabs/TSAD-Evaluator/master/examples/machine_temp/lstm_ad.real")

    
ranges_pred = get_ranges(df_pred, "0")
ranges_real = get_ranges(df_real, "0")

predicted_anomaly_ranges = ranges_pred
real_anomaly_range = ranges_real



In [477]:
# # Make a synthesized dataframe for gt and predict

# df_PSG = pd.DataFrame({
#     'linetime': pd.date_range(start='2017-01-04 1:00:00', end = '2017-01-04 3:00:00', freq='30S')
# })

# df_predict = pd.DataFrame({
#     'linetime': pd.date_range(start='2017-01-04 1:00:00', end = '2017-01-04 3:00:00', freq='30S')
# })

# """
# # Add labels wake:0/sleep:1 --> lets assume sleep labels are anomalies!
# # Adding 2 sleep periods [1:30_1:50, 2:30_2:45]
# """

# df_PSG ['w/s'] = 0
# df_predict ['w/s'] = 0

# #define sleep periods for PSG
# i = 60
# j = 100
# for c in range(i, j+1):
#     df_PSG.at[c, 'w/s'] = 1
    
# i = 180
# j = 210
# for c in range(i, j+1):
#     df_PSG.at[c, 'w/s'] = 1
# # df_PSG.loc[99:103]


# # define sleep periods for prediction
# i = 70
# j = 110
# for c in range(i, j+1):
#     df_predict.at[c, 'w/s'] = 1

# i = 170
# j = 200
# for c in range(i, j+1):
#     df_predict.at[c, 'w/s'] = 1
    


In [478]:
# import matplotlib.pyplot as plt

# plt.plot(df_PSG['linetime'], df_PSG['w/s'])
# plt.plot(df_predict['linetime'], df_predict['w/s'])

In [479]:
# predicted_anomaly_ranges = get_ranges(df_predict, 'w/s')
# real_anomaly_range = get_ranges(df_PSG, 'w/s')


In [480]:
#assuming that we have no bias
# bias = 1
def positional_bias(i, anomaly_len, kind = "flat"):
    if(kind == "flat"):
        return 1
    elif(kind == "front"):
        return anomaly_len - i +1
    elif(kind == "back"):
        return i
    elif(kind == "middle"):
        if (i <= anomaly_len/2 ):
            return i
        else:
            return anomaly_len - i +1
        

def w(r, overlap_set, bias_kind = "flat"):

    max_val = 0
    my_val = 0
    
    for i in range(len(r)):
        bias = positional_bias(i, len(r), kind = bias_kind)
        max_val = max_val + bias
        if(r[i] in overlap_set):
            my_val = my_val + bias
            
    return my_val/max_val
    #usage: w([2,3,4,5], [4,5]) --> returns 0.5 

    
    
#lets assume gamma function in the simplest way
def gamma(x):
    if(x != 0):
        return 1/x



def cardinality_factor(r, predicted_anomaly_ranges):
    x = 0
    
    if(r in predicted_anomaly_ranges): # the prediction is exactly the same as real
        return 1
    
    else:
        #  lets figure out that how many predicted ranges including r elements
        for predict in predicted_anomaly_ranges:
            if( all(item in r for item in predict) ): #the r is the subset of one predioction period
                return 1
            if (any(i in r for i in predict) ):
                x += 1
        if(x == 0):
            return 0
        else:
            return gamma(x)
        #usage: cardinality_factor([2,3,4], [[1,2], [3,5,6], [0,8]]) --> returns 0.5
    

def overlap(r, p):
    overlap_set = list(set(p) & set(r)) 
    
    return overlap_set #return an empty list if it doesn't found any element in common
    
    

def overlap_reward(r, predicted_anomaly_ranges):
    cardinality = cardinality_factor(r, predicted_anomaly_ranges)
    sum_weight = 0
    
    for predict in predicted_anomaly_ranges:
        
        overlap_set = overlap(r, predict) 
        weight = w(r, overlap_set, bias_kind = "flat")
#         weight = w(r, overlap_set, bias_kind = "front")
        sum_weight = sum_weight + weight 
        
    return cardinality*sum_weight
    #usage overlap_reward([2,3,4,0,8], [[1,2], [3,5,6], [0,8]]) --> returns 0.26
    

In [481]:
def existence_reward(r, predicted_anomaly_ranges):
    for predict in predicted_anomaly_ranges:
        if (any(i in r for i in predict) ):
                return 1
    return 0
        
        
alpha = 0.5 # the importance of rewarding existence


def recall(real_anomaly_range, predicted_anomaly_ranges, alpha= 0.5):
    total_recall = 0 
    for r in real_anomaly_range:
        recall_t = ( alpha*existence_reward(r, predicted_anomaly_ranges) ) + ( (1-alpha)*overlap_reward(r, predicted_anomaly_ranges))     
        total_recall = total_recall + recall_t
        
    return total_recall/len(real_anomaly_range)
    # usage: recall([[1,2,3,4], [10,11,12]], [[1,2], [3,5,6], [8,9], [10,11]], alpha = 0.5) --> returns 0.17
        
        
        

In [482]:
recall(real_anomaly_range, predicted_anomaly_ranges, alpha= 0)


0.5

In [483]:
# Desired output:

# (base) [21:22:13 donna:~/github/TSAD-Evaluator/src]$ ./evaluate -v -t ../examples/simple/simple.real ../examples/simple/simple.pred 1 0 reciprocal flat front
# Real Anomalies:
# [20, 21]
# [25, 27]
# [44, 47]
# Predicted Anomalies:
# [19, 20]
# [23, 30]
# [37, 37]
# [49, 49]
# Precision = 0.21875
# Recall = 0.555556
# F-Score = 0.313901


# recall(ranges_real, ranges_pred, alpha = 0.5)


# Precision 

In [485]:
def precision(real_anomaly_range, predicted_anomaly_ranges):
    total_precision = 0 
    for p in predicted_anomaly_ranges:
        precision_t = overlap_reward(p, real_anomaly_range)    
        total_precision = total_precision + precision_t 
        
    return total_precision/len(predicted_anomaly_ranges)




In [486]:
precision(real_anomaly_range, predicted_anomaly_ranges)


0.2916666666666667

In [490]:
real_anomaly_range = [[1,2,3], [10,11,12]]
predicted_anomaly_ranges = [[1,2,3], [4,5,6]]

precision(real_anomaly_range, predicted_anomaly_ranges)

0.5